In [1]:
!pwd

/Users/qook/Coding/finance_ml


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# R로 그대로 사용해서 데이터 생성 후 불러옴 - R에서는 인덱스가 날짜였는데 저장할 때 날짜는 저장이 안됐음!
# spy - 미국 주식, iev - 유럽 주식, ewj - 일본 주식, eem - 이머징 주식, tlt - 미국 장기채, ief - 미국 중기채
# iyr - 미국 리츠, rwx - 글로벌 리츠, gld - 금, dbc - 상품
data_price = pd.read_csv("etf_price.csv")
data_return = pd.read_csv("etf_return.csv")

In [4]:
# 첫 열이 기존 데이터의 인덱스 값을 그대로 저장했기 때문에 제거
data_price = data_price.iloc[:,1:]
data_return = data_return.iloc[:,1:]

In [5]:
# return's corr matrix
corr = data_return.corr()
corr.style.background_gradient(cmap='coolwarm')

,SPY,IEV,EWJ,EEM,TLT,IEF,IVR,RWX,GLD,DBC
SPY,1.000000,0.861627,0.729179,0.824976,-0.462804,-0.445944,0.320171,0.767590,0.043309,0.486981
IEV,0.861627,1.000000,0.734433,0.842473,-0.442253,-0.418202,0.298446,0.850298,0.116321,0.529564
EWJ,0.729179,0.734433,1.000000,0.705158,-0.332834,-0.316846,0.244743,0.718740,0.059109,0.394330
EEM,0.824976,0.842473,0.705158,1.000000,-0.376428,-0.354414,0.270180,0.805712,0.160782,0.544976
TLT,-0.462804,-0.442253,-0.332834,-0.376428,1.000000,0.922323,-0.063112,-0.323549,0.188187,-0.274902
IEF,-0.445944,-0.418202,-0.316846,-0.354414,0.922323,1.000000,-0.046063,-0.294197,0.236751,-0.254418
IVR,0.320171,0.298446,0.244743,0.270180,-0.063112,-0.046063,1.000000,0.338710,0.033623,0.168029
RWX,0.767590,0.850298,0.718740,0.805712,-0.323549,-0.294197,0.338710,1.000000,0.166720,0.494101
GLD,0.043309,0.116321,0.059109,0.160782,0.188187,0.236751,0.033623,0.166720,1.000000,0.314970
DBC,0.486981,0.529564,0.394330,0.544976,-0.274902,-0.254418,0.168029,0.494101,0.314970,1.000000


# 최소분산 포트폴리오

### slsqp 알고리즘을 이용

In [6]:
from scipy.optimize import minimize

In [7]:
cov = data_return.cov()

In [8]:
# 목적함수 정의, w는 가중치 array(n X 1)
def objective(w):
    obj = w.reshape([1,-1]) @ cov @ w
    return obj

In [9]:
# 제약함수 정의, w>=0 
def con_ineq(w):
    return(w)

In [10]:
# 제약함수 정의, w합 = 1 -> w합 -1 = 0
def con_eq(w):
    sum_w = sum(w)
    return(sum_w - 1)

In [11]:
con = [{'type' : 'ineq', 'fun' : con_ineq}, {'type' : 'eq', 'fun' : con_eq}]

In [12]:
# slsqp 알고리즘을 이용하고 제약조건을 걸어서 최소값을 찾아줌
min_cov = minimize(objective, np.ones(cov.shape[0]), method='SLSQP', constraints=con)

In [13]:
#최소값 0.000058, 가중치 - x array 값, 합해서 1이 나옴
min_cov.x.round(4)

array([0.0999, 0.0995, 0.0999, 0.0994, 0.1014, 0.1013, 0.0981, 0.0997,
       0.1007, 0.1002])

## qp 알고리즘 이용

In [14]:
# 필요 라이브러리 설치
# !pip install CVXOPT 

In [15]:
from cvxopt import solvers, matrix

## solvers 설명을 보고 해당 조건에 맞는 행렬을 책과 비교해서 값을 찾아서 대입

In [16]:
P = matrix(cov.values)
q = matrix(np.zeros(cov.shape[0]))

In [17]:
a = np.ones(cov.shape[0])
a = a.reshape([1,-1])
b =np.eye(cov.shape[0])
c = -np.eye(cov.shape[0])
G = matrix(-np.concatenate((a,b,c), axis = 0))

In [18]:
h1 = np.ones(1).reshape([-1,1])
h2 = np.zeros(cov.shape[0]).reshape([-1,1])
h3 = -np.ones(cov.shape[0]).reshape([-1,1])
h = matrix(-np.concatenate((h1,h2,h3), axis = 0))

In [19]:
# 결과 보는 법을 모르겠다. - x가 원하는 정답
solvers.qp(P,q,G,h)

     pcost       dcost       gap    pres   dres
 0:  8.1380e-05 -1.1890e+01  1e+01  3e-17  6e+00
 1:  8.1363e-05 -1.1939e-01  1e-01  1e-16  6e-02
 2:  7.9697e-05 -1.6755e-03  2e-03  2e-16  8e-04
 3:  4.0812e-05 -2.0590e-04  2e-04  1e-16  1e-04
 4:  1.4465e-05 -2.9518e-05  4e-05  2e-16  3e-06
 5:  5.6017e-06  1.5586e-07  5e-06  2e-16  3e-07
 6:  4.4140e-06  3.3048e-06  1e-06  2e-16  4e-08
 7:  4.0225e-06  3.8892e-06  1e-07  2e-16  4e-20
 8:  3.9679e-06  3.9619e-06  6e-09  2e-16  1e-19
Optimal solution found.


{'x': <10x1 matrix, tc='d'>,
 'y': <0x1 matrix, tc='d'>,
 's': <21x1 matrix, tc='d'>,
 'z': <21x1 matrix, tc='d'>,
 'status': 'optimal',
 'gap': 5.942927957238514e-09,
 'relative gap': 0.0015000074993498978,
 'primal objective': 3.9678750915058685e-06,
 'dual objective': 3.961932163548632e-06,
 'primal infeasibility': 2.2688987142716445e-16,
 'dual infeasibility': 9.819738844077818e-20,
 'primal slack': 1.1403250788138018e-05,
 'dual slack': 8.519195125908161e-11,
 'iterations': 8}

In [20]:
# 가중치 결과
qp = solvers.qp(P,q,G,h)
print(qp['x'])

     pcost       dcost       gap    pres   dres
 0:  8.1380e-05 -1.1890e+01  1e+01  3e-17  6e+00
 1:  8.1363e-05 -1.1939e-01  1e-01  1e-16  6e-02
 2:  7.9697e-05 -1.6755e-03  2e-03  2e-16  8e-04
 3:  4.0812e-05 -2.0590e-04  2e-04  1e-16  1e-04
 4:  1.4465e-05 -2.9518e-05  4e-05  2e-16  3e-06
 5:  5.6017e-06  1.5586e-07  5e-06  2e-16  3e-07
 6:  4.4140e-06  3.3048e-06  1e-06  2e-16  4e-08
 7:  4.0225e-06  3.8892e-06  1e-07  2e-16  4e-20
 8:  3.9679e-06  3.9619e-06  6e-09  2e-16  1e-19
Optimal solution found.
[ 1.40e-01]
[ 2.86e-03]
[ 2.31e-02]
[ 1.76e-04]
[ 1.14e-05]
[ 7.71e-01]
[ 6.70e-05]
[ 6.48e-04]
[ 2.03e-04]
[ 6.14e-02]



## 포트폴리오 최적화를 해주는 패키지를 사용해서 구하기 - 보류

## 최소 및 최대 투자비중 제약조건 -> bound 설정

## 각 자산별 제약조건의 추가 -> qp에서 h값만 바꿔준다

In [21]:
print(qp['x'])

[ 1.40e-01]
[ 2.86e-03]
[ 2.31e-02]
[ 1.76e-04]
[ 1.14e-05]
[ 7.71e-01]
[ 6.70e-05]
[ 6.48e-04]
[ 2.03e-04]
[ 6.14e-02]



# 최대분산 포트폴리오 

### QP 알고리즘 이용 - 마찬가지로 책의 조건을 해당 함수의 조건에 맞게 변형해서 사용

In [22]:
P = matrix(cov.values)
q = matrix(np.zeros(cov.shape[0]))

In [23]:
A = -np.diag(cov).reshape(1,-1)
G = -np.eye((cov.shape[0]))
G = matrix(np.concatenate((A,G), axis = 0))
G

<11x10 matrix, tc='d'>

In [24]:
h1 = np.zeros((cov.shape[0])).reshape(-1,1)
h = matrix(np.concatenate((np.array(1).reshape(1,1),h1), axis = 0))

In [25]:
# 결과 보는 법을 모르겠다. - x가 원하는 정답
solvers.qp(P = P,q = q,G = G,h = h)

     pcost       dcost       gap    pres   dres
 0:  5.6663e-10 -9.9594e-01  2e+01  3e+00  6e+00
 1:  2.8710e-03 -4.5922e-01  5e-01  3e-02  6e-02
 2:  2.4757e-03 -9.4234e-03  1e-02  5e-04  1e-03
 3:  5.3894e-04 -5.9303e-04  1e-03  4e-06  8e-06
 4:  7.9511e-05 -8.0957e-05  2e-04  3e-16  2e-19
 5:  1.1603e-05 -1.1825e-05  2e-05  4e-16  5e-20
 6:  1.6931e-06 -1.7255e-06  3e-06  1e-16  2e-20
 7:  2.4706e-07 -2.5177e-07  5e-07  5e-17  8e-21
 8:  3.6049e-08 -3.6738e-08  7e-08  2e-16  3e-21
Optimal solution found.


{'x': <10x1 matrix, tc='d'>,
 'y': <0x1 matrix, tc='d'>,
 's': <11x1 matrix, tc='d'>,
 'z': <11x1 matrix, tc='d'>,
 'status': 'optimal',
 'gap': 7.278689866015878e-08,
 'relative gap': None,
 'primal objective': 3.6049128578662716e-08,
 'dual objective': -3.673777008149605e-08,
 'primal infeasibility': 2.0050825139043588e-16,
 'dual infeasibility': 3.47319418261349e-21,
 'primal slack': 0.002762079655063053,
 'dual slack': 6.886415028333234e-10,
 'iterations': 8}

In [26]:
solv = solvers.qp(P = P,q = q,G = G,h = h)
np.array(solv['x'])

     pcost       dcost       gap    pres   dres
 0:  5.6663e-10 -9.9594e-01  2e+01  3e+00  6e+00
 1:  2.8710e-03 -4.5922e-01  5e-01  3e-02  6e-02
 2:  2.4757e-03 -9.4234e-03  1e-02  5e-04  1e-03
 3:  5.3894e-04 -5.9303e-04  1e-03  4e-06  8e-06
 4:  7.9511e-05 -8.0957e-05  2e-04  3e-16  2e-19
 5:  1.1603e-05 -1.1825e-05  2e-05  4e-16  5e-20
 6:  1.6931e-06 -1.7255e-06  3e-06  1e-16  2e-20
 7:  2.4706e-07 -2.5177e-07  5e-07  5e-17  8e-21
 8:  3.6049e-08 -3.6738e-08  7e-08  2e-16  3e-21
Optimal solution found.


array([[0.00401054],
       [0.00354753],
       [0.00395522],
       [0.00337884],
       [0.00823036],
       [0.01351492],
       [0.00276208],
       [0.00361451],
       [0.00460873],
       [0.00428457]])

In [27]:
# 비율 표준화
np.array(solv['x']) / np.sum(np.array(solv['x']))

array([[0.07726351],
       [0.06834359],
       [0.07619766],
       [0.06509376],
       [0.15855887],
       [0.26036646],
       [0.05321177],
       [0.06963398],
       [0.08878773],
       [0.08254266]])

## 포트폴리오 최적 패키지를 사용 - 보류

## 최소 및 최대 투자비중 제약조건 & 자산별 제약조건 추가 - 제약식만 바꿔주면 됨

## 위험균형 포트폴리오

#### 위험기여도

In [28]:
def get_RC(w, covmat):
    port_vol = np.transpose(w) @ covmat @ w
    port_std = np.sqrt(port_vol)
    
    MRC = (covmat @ w) / port_std[0]   #### dataframe의 원소를 갖고 와야함 ㅎ
    RC = (MRC) * w
    RC = RC / np.sum(RC)
    
    return RC

In [29]:
ret_stock_bond = data_return.loc[:,['SPY','TLT']]

In [30]:
cov_stock_bond = ret_stock_bond.cov()
cov_stock_bond

,SPY,TLT
SPY,0.000118,-0.000046
TLT,-0.000046,0.000085


In [31]:
RC_stock_bond = get_RC(np.array([0.6,0.4]).reshape(-1,1), cov_stock_bond)

In [32]:
RC_stock_bond

,0
SPY,0.924736
TLT,0.075264


### rp 함수를 이용한 최적화 & 위험예산 포트폴리오 - 보류

## 인덱스 포트폴리오 구성하기

### 인덱스 포트폴리오 복제하기 - 시가총액 값이 "" 까지 포함되서 문자로 불러와져서 시가총액 정렬이 안됨 - 방법 고민 필요! - 함수 만들어버림

In [54]:
KOR_ticker = pd.read_csv("KOR_ticker.csv")

In [55]:
# 문자로 된 숫자를 숫자로 바꿔주기 위해 양끝의 " 제거를 위해 양 끝 값 빼고 추출 - "숫자" 형태로 문자형식으로 되어 있음
def to_num(word):
    return word[1:-1]

In [56]:
KOR_ticker['시가총액'] = KOR_ticker['시가총액'].apply(to_num)
KOR_ticker

,종목코드,종목명,종가,대비,등락률,EPS,PER,BPS,PBR,주당배당금,배당수익률,시장구분,업종명,시가총액
0,"""060310""","""3S""","""2400""","""5""","""0.21""",NaN,NaN,"""745""","""3.22""","""0""","""0.00""","""KOSDAQ""","""기계·장비""",107526026400
1,"""095570""","""AJ네트웍스""","""4000""","""30""","""0.76""","""982""","""4.07""","""6802""","""0.59""","""300""","""7.50""","""KOSPI""","""서비스업""",187289180000
2,"""006840""","""AK홀딩스""","""28700""","""50""","""0.17""","""2168""","""13.24""","""62448""","""0.46""","""750""","""2.61""","""KOSPI""","""기타금융""",380205000700
3,"""054620""","""APS홀딩스""","""7710""","""150""","""1.98""",NaN,NaN,"""10530""","""0.73""","""0""","""0.00""","""KOSDAQ""","""금융""",157239443910
4,"""265520""","""AP시스템""","""29000""","""550""","""1.93""","""671""","""43.22""","""7468""","""3.88""","""50""","""0.17""","""KOSDAQ""","""반도체""",419926583000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2181,"""189980""","""흥국에프엔비""","""4485""","""1035""","""30.00""","""151""","""29.70""","""1745""","""2.57""","""30""","""0.67""","""KOSDAQ""","""음식료·담배""",172166264595
2182,"""000540""","""흥국화재""","""3860""","""-10""","""-0.26""","""510""","""7.57""","""10926""","""0.35""","""0""","""0.00""","""KOSPI""","""보험""",247976609700
2183,"""003280""","""흥아해운""","""258""","""0""","""0.00""",NaN,NaN,"""276""","""0.93""","""0""","""0.00""","""KOSPI""","""운수창고업""",30117000222
2184,"""037440""","""희림""","""4290""","""0""","""0.00""","""288""","""14.90""","""3827""","""1.12""","""150""","""3.50""","""KOSDAQ""","""기타서비스""",59727417750


In [57]:
KOSPI200 = KOR_ticker[KOR_ticker['시장구분'] == '"KOSPI"']
KOSPI200_long = KOSPI200['시가총액'].astype('long')

In [58]:
# 시가총액을 기준으로 200개 추출
KOSPI200 = KOSPI200.loc[list(KOSPI200_long.sort_values(ascending=False).index)[:200]]
KOSPI200

,종목코드,종목명,종가,대비,등락률,EPS,PER,BPS,PBR,주당배당금,배당수익률,시장구분,업종명,시가총액
779,"""005930""","""삼성전자""","""82800""","""800""","""0.98""","""3166""","""26.15""","""37528""","""2.21""","""1416""","""1.71""","""KOSPI""","""전기전자""",494297995140000
170,"""000660""","""SK하이닉스""","""140000""","""3000""","""2.19""","""2943""","""47.57""","""65836""","""2.13""","""1000""","""0.71""","""KOSPI""","""전기전자""",101920331100000
110,"""051910""","""LG화학""","""944000""","""5000""","""0.53""","""4085""","""231.09""","""217230""","""4.35""","""2000""","""0.21""","""KOSPI""","""화학""",66639171792000
119,"""035420""","""NAVER""","""380500""","""7000""","""1.87""","""4006""","""94.98""","""35223""","""10.80""","""376""","""0.10""","""KOSPI""","""서비스업""",62502221797500
2125,"""005380""","""현대차""","""232500""","""4000""","""1.75""","""11310""","""20.56""","""253001""","""0.92""","""4000""","""1.72""","""KOSPI""","""운수장비""",49677853477500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1919,"""103140""","""풍산""","""32900""","""450""","""1.39""","""629""","""52.31""","""49306""","""0.67""","""400""","""1.22""","""KOSPI""","""철강금속""",921998746200
2039,"""020000""","""한섬""","""37300""","""100""","""0.27""","""3881""","""9.61""","""42450""","""0.88""","""450""","""1.21""","""KOSPI""","""섬유의복""",918699000000
430,"""005880""","""대한해운""","""3700""","""-45""","""-1.20""","""374""","""9.89""","""3394""","""1.09""","""0""","""0.00""","""KOSPI""","""운수창고업""",903803033000
586,"""071840""","""롯데하이마트""","""38250""","""0""","""0.00""",NaN,NaN,"""81410""","""0.47""","""1000""","""2.61""","""KOSPI""","""유통업""",902994984000


In [59]:
# 시가총액비중 추가
KOSPI200['시가총액비중'] = KOSPI200['시가총액'].astype('long') / sum(KOSPI200['시가총액'].astype('long'))
KOSPI200

,종목코드,종목명,종가,대비,등락률,EPS,PER,BPS,PBR,주당배당금,배당수익률,시장구분,업종명,시가총액,시가총액비중
779,"""005930""","""삼성전자""","""82800""","""800""","""0.98""","""3166""","""26.15""","""37528""","""2.21""","""1416""","""1.71""","""KOSPI""","""전기전자""",494297995140000,0.264562
170,"""000660""","""SK하이닉스""","""140000""","""3000""","""2.19""","""2943""","""47.57""","""65836""","""2.13""","""1000""","""0.71""","""KOSPI""","""전기전자""",101920331100000,0.054551
110,"""051910""","""LG화학""","""944000""","""5000""","""0.53""","""4085""","""231.09""","""217230""","""4.35""","""2000""","""0.21""","""KOSPI""","""화학""",66639171792000,0.035667
119,"""035420""","""NAVER""","""380500""","""7000""","""1.87""","""4006""","""94.98""","""35223""","""10.80""","""376""","""0.10""","""KOSPI""","""서비스업""",62502221797500,0.033453
2125,"""005380""","""현대차""","""232500""","""4000""","""1.75""","""11310""","""20.56""","""253001""","""0.92""","""4000""","""1.72""","""KOSPI""","""운수장비""",49677853477500,0.026589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1919,"""103140""","""풍산""","""32900""","""450""","""1.39""","""629""","""52.31""","""49306""","""0.67""","""400""","""1.22""","""KOSPI""","""철강금속""",921998746200,0.000493
2039,"""020000""","""한섬""","""37300""","""100""","""0.27""","""3881""","""9.61""","""42450""","""0.88""","""450""","""1.21""","""KOSPI""","""섬유의복""",918699000000,0.000492
430,"""005880""","""대한해운""","""3700""","""-45""","""-1.20""","""374""","""9.89""","""3394""","""1.09""","""0""","""0.00""","""KOSPI""","""운수창고업""",903803033000,0.000484
586,"""071840""","""롯데하이마트""","""38250""","""0""","""0.00""",NaN,NaN,"""81410""","""0.47""","""1000""","""2.61""","""KOSPI""","""유통업""",902994984000,0.000483


In [60]:
KOSPI200['매수금액'] = 100000000 * KOSPI200['시가총액비중'] 

In [61]:
# 종가도 숫자로 바꿔줌
KOSPI200['종가'] = KOSPI200['종가'].apply(to_num)
KOSPI200['매수주수'] = KOSPI200['매수금액'] / KOSPI200['종가'].astype('long')

In [62]:
# 매수주수 내림
KOSPI200['매수주수'] = KOSPI200['매수주수'].apply(np.floor)

In [63]:
# 실제매수금액
sum(KOSPI200['종가'].astype('long') * KOSPI200['매수주수'])

88114760.0

### 팩터를 이용한 인핸스드 포트폴리오 구성하기 - 시각화는 제외

#### 단순가감법

In [133]:
KOSPI200_p = KOSPI200.loc[:,['종목명','PBR','시가총액비중']]

In [134]:
KOSPI200_p = KOSPI200_p.sort_values(by='PBR', ascending=False)

In [135]:
KOSPI200_p = KOSPI200_p.dropna()

In [136]:
KOSPI200_p

,종목명,PBR,시가총액비중
1277,"""영진약품""","""9.91""",0.000614
440,"""더존비즈온""","""9.10""",0.001680
2060,"""한올바이오파마""","""8.58""",0.000644
1231,"""엔씨소프트""","""8.21""",0.010987
1683,"""카카오""","""7.94""",0.022836
...,...,...,...
27,"""DGB금융지주""","""0.29""",0.000712
277,"""기업은행""","""0.27""",0.003542
8,"""BNK금융지주""","""0.25""",0.001120
2086,"""한화생명""","""0.22""",0.001478


In [137]:
KOSPI200_p["PBR"] = KOSPI200_p["PBR"].apply(to_num).astype('float')

In [142]:
# PBR이 작은 값들에는 비중을 더해주고 큰 값들에는 비중을 줄여줌
KOSPI200_p['조절비중'] = 0 
for i in range(len(KOSPI200_p)):
    if i > 100:
        KOSPI200_p['조절비중'].iloc[i] = (KOSPI200_p["시가총액비중"].iloc[i] + 0.0005)
    else:
        KOSPI200_p['조절비중'].iloc[i] = (KOSPI200_p["시가총액비중"].iloc[i] - 0.0005)

/Users/qook/Coding/Finance_ML/venv/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [144]:
KOSPI200_p['조절비중'] = KOSPI200_p['조절비중'] / sum(KOSPI200_p['조절비중'])

In [146]:
KOSPI200_p['차이'] = KOSPI200_p['조절비중'] - KOSPI200_p['시가총액비중']

In [147]:
KOSPI200_p

,종목명,PBR,시가총액비중,조절비중,차이
1277,"""영진약품""",9.91,0.000614,0.000115,-0.000499
440,"""더존비즈온""",9.10,0.001680,0.001191,-0.000490
2060,"""한올바이오파마""",8.58,0.000644,0.000146,-0.000499
1231,"""엔씨소프트""",8.21,0.010987,0.010580,-0.000407
1683,"""카카오""",7.94,0.022836,0.022535,-0.000301
...,...,...,...,...,...
27,"""DGB금융지주""",0.29,0.000712,0.001223,0.000511
277,"""기업은행""",0.27,0.003542,0.004078,0.000536
8,"""BNK금융지주""",0.25,0.001120,0.001634,0.000514
2086,"""한화생명""",0.22,0.001478,0.001996,0.000518


#### 팩터에 대한 전체 종목의 틸트 - 보류
#### R에서 pnorm() = python에서 scipy.stat.normal.cdf()